In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

print("🚀 Starting Bureau Feature Engineering...")

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("\n📂 Loading data...")

app_train = pd.read_csv('DATA/application_train.csv', 
                        usecols=['SK_ID_CURR', 'TARGET', 'AMT_INCOME_TOTAL', 'AMT_ANNUITY'])
bureau = pd.read_csv('DATA/bureau.csv', 
                     usecols=['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE'])
bureau_balance = pd.read_csv('DATA/bureau_balance.csv', 
                             usecols=['SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS'])

print(f"✅ Customers: {len(app_train):,}")
print(f"✅ Bureau records: {len(bureau):,}")
print(f"✅ Bureau balance: {len(bureau_balance):,}")

# ============================================================================
# STEP 2: STATUS → DPD MAPPING
# ============================================================================
print("\n🔄 Converting STATUS to DPD...")

status_map = {'C': 0, 'X': 0, '0': 0, '1': 15, '2': 30, '3': 60, '4': 90, '5': 120}
bureau_balance['DPD'] = bureau_balance['STATUS'].map(status_map).fillna(0)

# ============================================================================
# STEP 3: TIME FILTERING
# ============================================================================
print("📅 Filtering time periods...")

bb_6m = bureau_balance[bureau_balance['MONTHS_BALANCE'] >= -6].copy()
bb_3m = bureau_balance[bureau_balance['MONTHS_BALANCE'] >= -3].copy()

# ============================================================================
# STEP 4: CALCULATE DPD COUNTS (Loan Level)
# ============================================================================
print("\n🧮 Calculating DPD features...")

# 6-month features
bureau_6m = bb_6m.groupby('SK_ID_BUREAU').agg(
    dpd_15_count_6m=('DPD', lambda x: (x >= 15).sum()),
    dpd_30_count_6m=('DPD', lambda x: (x >= 30).sum()),
    dpd_90_count_6m=('DPD', lambda x: (x >= 90).sum()),
    max_dpd_6m=('DPD', 'max')
).reset_index()

# 3-month features  
bureau_3m = bb_3m.groupby('SK_ID_BUREAU').agg(
    dpd_30_count_3m=('DPD', lambda x: (x >= 30).sum())
).reset_index()

print(f"   Processed {len(bureau_6m):,} loan histories")

# ============================================================================
# STEP 5: MERGE TO BUREAU
# ============================================================================
print("\n🔗 Merging features...")

bureau_enriched = bureau.merge(bureau_6m, on='SK_ID_BUREAU', how='left')
bureau_enriched = bureau_enriched.merge(bureau_3m, on='SK_ID_BUREAU', how='left')
bureau_enriched = bureau_enriched.fillna(0)

# ============================================================================
# STEP 6: AGGREGATE TO CUSTOMER LEVEL
# ============================================================================
print("👤 Aggregating to customer level...")

customer_bureau = bureau_enriched.groupby('SK_ID_CURR').agg(
    active_loans_count=('SK_ID_BUREAU', 'count'),
    dpd_15_count_6m=('dpd_15_count_6m', 'sum'),
    dpd_30_count_6m=('dpd_30_count_6m', 'sum'),
    dpd_90_count_6m=('dpd_90_count_6m', 'sum'),
    max_dpd_6m=('max_dpd_6m', 'max'),
    dpd_30_count_3m=('dpd_30_count_3m', 'sum')
).reset_index()

print(f"   {len(customer_bureau):,} customers with bureau data")

# ============================================================================
# STEP 7: CREATE RISK FLAGS
# ============================================================================
print("\n🚦 Creating risk flags...")

customer_bureau['hard_reject_flag'] = (customer_bureau['dpd_90_count_6m'] > 0).astype(int)

def assign_risk(row):
    if row['dpd_90_count_6m'] > 0:
        return 'HIGH'
    elif row['dpd_30_count_6m'] > 2:
        return 'HIGH'
    elif row['dpd_30_count_3m'] > 0:
        return 'MEDIUM'
    elif row['dpd_15_count_6m'] > 3:
        return 'MEDIUM'
    return 'LOW'

customer_bureau['bureau_risk_flag'] = customer_bureau.apply(assign_risk, axis=1)

# ============================================================================
# STEP 8: MERGE WITH CUSTOMER MASTER
# ============================================================================
print("\n🔄 Final merge...")

final_df = app_train.rename(columns={'SK_ID_CURR': 'customer_id'}).merge(
    customer_bureau.rename(columns={'SK_ID_CURR': 'customer_id'}),
    on='customer_id',
    how='left'
)

# Fill missing values
fill_values = {
    'active_loans_count': 0,
    'dpd_15_count_6m': 0,
    'dpd_30_count_6m': 0,
    'dpd_90_count_6m': 0,
    'max_dpd_6m': 0,
    'dpd_30_count_3m': 0,
    'hard_reject_flag': 0,
    'bureau_risk_flag': 'LOW'
}
final_df = final_df.fillna(fill_values)

# Convert types
final_df['active_loans_count'] = final_df['active_loans_count'].astype(int)
final_df['hard_reject_flag'] = final_df['hard_reject_flag'].astype(int)

# ============================================================================
# STEP 9: STATISTICS & VALIDATION
# ============================================================================
print("\n" + "="*60)
print("✅ BUREAU FEATURES COMPLETE!")
print("="*60)

print(f"\n📊 Dataset: {final_df.shape}")
print(f"   Total customers: {len(final_df):,}")
print(f"   With bureau history: {(final_df['active_loans_count'] > 0).sum():,}")

print(f"\n📈 Risk Distribution:")
for risk, count in final_df['bureau_risk_flag'].value_counts().items():
    print(f"   {risk:8s}: {count:6,} ({count/len(final_df)*100:5.1f}%)")

print(f"\n⚠️  Hard Rejects: {final_df['hard_reject_flag'].sum():,}")

print(f"\n🔢 Feature Summary:")
print(final_df[['active_loans_count', 'dpd_15_count_6m', 'dpd_30_count_6m', 
               'dpd_90_count_6m']].describe())

# ============================================================================
# STEP 10: SAVE
# ============================================================================
output_path = Path('DATA/processed')
output_path.mkdir(exist_ok=True)

output_file = output_path / 'customer_bureau_features.csv'
final_df.to_csv(output_file, index=False)

print(f"\n💾 Saved: {output_file}")
print(f"   Columns: {final_df.shape[1]}")
print("\n✅ Ready for Step 4: Installment Payment Analysis")

🚀 Starting Bureau Feature Engineering...

📂 Loading data...
✅ Customers: 307,511
✅ Bureau records: 1,716,428
✅ Bureau balance: 27,299,925

🔄 Converting STATUS to DPD...
📅 Filtering time periods...

🧮 Calculating DPD features...
   Processed 653,874 loan histories

🔗 Merging features...
👤 Aggregating to customer level...
   305,811 customers with bureau data

🚦 Creating risk flags...

🔄 Final merge...

✅ BUREAU FEATURES COMPLETE!

📊 Dataset: (307511, 12)
   Total customers: 307,511
   With bureau history: 263,491

📈 Risk Distribution:
   LOW     : 304,558 ( 99.0%)
   MEDIUM  :  1,905 (  0.6%)
   HIGH    :  1,048 (  0.3%)

⚠️  Hard Rejects: 926

🔢 Feature Summary:
       active_loans_count  dpd_15_count_6m  dpd_30_count_6m  dpd_90_count_6m
count       307511.000000    307511.000000    307511.000000    307511.000000
mean             4.765114         0.101457         0.026093         0.018985
std              4.496199         0.777261         0.510909         0.465391
min              0.00

In [5]:
import pandas as pd

# Load the created file
bureau_df = pd.read_csv('DATA/processed/customer_bureau_features.csv')

print("✅ File loaded successfully!")
print(f"\n📊 Shape: {bureau_df.shape}")
print(f"   Rows: {bureau_df.shape[0]:,}")
print(f"   Columns: {bureau_df.shape[1]}")

print(f"\n📋 Columns:")
for i, col in enumerate(bureau_df.columns, 1):
    print(f"   {i:2d}. {col}")

print(f"\n🔍 First 5 rows:")
print(bureau_df.head())

print(f"\n📈 Risk Distribution:")
print(bureau_df['bureau_risk_flag'].value_counts())

print(f"\n⚠️ Hard Rejects: {bureau_df['hard_reject_flag'].sum():,}")

print(f"\n✅ Bureau features ready for next step!")

✅ File loaded successfully!

📊 Shape: (307511, 12)
   Rows: 307,511
   Columns: 12

📋 Columns:
    1. customer_id
    2. TARGET
    3. AMT_INCOME_TOTAL
    4. AMT_ANNUITY
    5. active_loans_count
    6. dpd_15_count_6m
    7. dpd_30_count_6m
    8. dpd_90_count_6m
    9. max_dpd_6m
   10. dpd_30_count_3m
   11. hard_reject_flag
   12. bureau_risk_flag

🔍 First 5 rows:
   customer_id  TARGET  AMT_INCOME_TOTAL  AMT_ANNUITY  active_loans_count  \
0       100002       1          202500.0      24700.5                   8   
1       100003       0          270000.0      35698.5                   4   
2       100004       0           67500.0       6750.0                   2   
3       100006       0          135000.0      29686.5                   0   
4       100007       0          121500.0      21865.5                   1   

   dpd_15_count_6m  dpd_30_count_6m  dpd_90_count_6m  max_dpd_6m  \
0              0.0              0.0              0.0         0.0   
1              0.0           

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

print("🚀 Starting Installment Payment Analysis...")

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("\n📂 Loading data...")

# Load previous bureau features
bureau_df = pd.read_csv('DATA/processed/customer_bureau_features.csv')

# Load installments data
installments = pd.read_csv('DATA/installments_payments.csv')

print(f"✅ Bureau features: {len(bureau_df):,} customers")
print(f"✅ Installment records: {len(installments):,}")

# ============================================================================
# STEP 2: CALCULATE DAYS LATE (Payment Discipline)
# ============================================================================
print("\n⏰ Calculating payment delays...")

# DAYS_INSTALMENT = when payment was due (negative = days before application)
# DAYS_ENTRY_PAYMENT = when payment was actually made
# days_late = actual - due (positive = late, negative = early)

installments['days_late'] = (
    installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']
)

# Handle missing payments (no DAYS_ENTRY_PAYMENT)
# If payment date is missing, it might be ongoing or unpaid
installments['days_late'] = installments['days_late'].fillna(0)

print(f"   Payment delay range: {installments['days_late'].min():.0f} to {installments['days_late'].max():.0f} days")

# ============================================================================
# STEP 3: CREATE DELAY BUCKETS
# ============================================================================
print("\n📊 Creating delay categories...")

# Categorize delays
installments['is_late_15'] = (installments['days_late'] > 15).astype(int)
installments['is_late_30'] = (installments['days_late'] > 30).astype(int)
installments['is_late_60'] = (installments['days_late'] > 60).astype(int)
installments['is_late_90'] = (installments['days_late'] > 90).astype(int)

# ============================================================================
# STEP 4: TIME-BASED FILTERING (Recent 6m & 3m)
# ============================================================================
print("\n📅 Filtering recent payments...")

# DAYS_INSTALMENT is negative (0 = most recent, -30 = 30 days ago, etc.)
# Last 6 months ≈ -180 days
# Last 3 months ≈ -90 days

inst_6m = installments[installments['DAYS_INSTALMENT'] >= -180].copy()
inst_3m = installments[installments['DAYS_INSTALMENT'] >= -90].copy()

print(f"   Last 6 months: {len(inst_6m):,} payments")
print(f"   Last 3 months: {len(inst_3m):,} payments")

# ============================================================================
# STEP 5: AGGREGATE PER LOAN (SK_ID_PREV)
# ============================================================================
print("\n🧮 Aggregating payment behavior per loan...")

def aggregate_payment_features(df, suffix):
    """Aggregate payment discipline features"""
    agg_df = df.groupby('SK_ID_PREV').agg(
        total_payments=(('NUM_INSTALMENT_NUMBER', 'count')),
        late_15_count=(('is_late_15', 'sum')),
        late_30_count=(('is_late_30', 'sum')),
        late_60_count=(('is_late_60', 'sum')),
        late_90_count=(('is_late_90', 'sum')),
        max_days_late=(('days_late', 'max')),
        avg_days_late=(('days_late', 'mean'))
    ).reset_index()
    
    # Add suffix to columns (except SK_ID_PREV)
    cols_to_rename = {col: f"{col}{suffix}" for col in agg_df.columns if col != 'SK_ID_PREV'}
    agg_df = agg_df.rename(columns=cols_to_rename)
    
    return agg_df

# Create 6-month features
inst_feat_6m = aggregate_payment_features(inst_6m, '_6m')

# Create 3-month features (only key metrics)
inst_feat_3m = inst_3m.groupby('SK_ID_PREV').agg(
    late_30_count_3m=(('is_late_30', 'sum')),
    late_90_count_3m=(('is_late_90', 'sum'))
).reset_index()

print(f"   Features created for {len(inst_feat_6m):,} loans")

# ============================================================================
# STEP 6: MAP TO CUSTOMERS (SK_ID_CURR)
# ============================================================================
print("\n🔗 Mapping loans to customers...")

# Get loan-to-customer mapping
prev_app = pd.read_csv('DATA/previous_application.csv', 
                       usecols=['SK_ID_PREV', 'SK_ID_CURR'])

# Merge installment features with customer IDs
inst_feat_6m = inst_feat_6m.merge(prev_app, on='SK_ID_PREV', how='left')
inst_feat_3m = inst_feat_3m.merge(prev_app, on='SK_ID_PREV', how='left')

# Remove loans without customer mapping
inst_feat_6m = inst_feat_6m.dropna(subset=['SK_ID_CURR'])
inst_feat_3m = inst_feat_3m.dropna(subset=['SK_ID_CURR'])

# ============================================================================
# STEP 7: AGGREGATE TO CUSTOMER LEVEL
# ============================================================================
print("\n👤 Aggregating to customer level...")

# 6-month customer features
customer_inst_6m = inst_feat_6m.groupby('SK_ID_CURR').agg(
    total_payments_6m=(('total_payments_6m', 'sum')),
    total_late_15_6m=(('late_15_count_6m', 'sum')),
    total_late_30_6m=(('late_30_count_6m', 'sum')),
    total_late_60_6m=(('late_60_count_6m', 'sum')),
    total_late_90_6m=(('late_90_count_6m', 'sum')),
    max_days_late_6m=(('max_days_late_6m', 'max')),
    avg_days_late_6m=(('avg_days_late_6m', 'mean'))
).reset_index()

# 3-month customer features
customer_inst_3m = inst_feat_3m.groupby('SK_ID_CURR').agg(
    total_late_30_3m=(('late_30_count_3m', 'sum')),
    total_late_90_3m=(('late_90_count_3m', 'sum'))
).reset_index()

print(f"   {len(customer_inst_6m):,} customers with payment history")

# ============================================================================
# STEP 8: CREATE PAYMENT DISCIPLINE FLAGS
# ============================================================================
print("\n🚦 Creating payment discipline flags...")

# Merge 6m and 3m features
customer_inst = customer_inst_6m.merge(customer_inst_3m, on='SK_ID_CURR', how='left')
customer_inst = customer_inst.fillna(0)

# Payment discipline flag
def assign_payment_discipline(row):
    if row['total_late_90_6m'] > 0:
        return 'POOR'
    elif row['total_late_30_3m'] > 2:
        return 'POOR'
    elif row['total_late_30_6m'] > 3:
        return 'MODERATE'
    elif row['total_late_15_6m'] > 5:
        return 'MODERATE'
    else:
        return 'GOOD'

customer_inst['payment_discipline_flag'] = customer_inst.apply(
    assign_payment_discipline, axis=1
)

# Recent stress indicator (last 3 months)
customer_inst['recent_payment_stress'] = (
    customer_inst['total_late_30_3m'] > 0
).astype(int)

# ============================================================================
# STEP 9: MERGE WITH BUREAU FEATURES
# ============================================================================
print("\n🔄 Merging with bureau features...")

# Rename to customer_id for consistency
customer_inst = customer_inst.rename(columns={'SK_ID_CURR': 'customer_id'})

# Merge
final_df = bureau_df.merge(customer_inst, on='customer_id', how='left')

# Fill customers with no installment history
fill_cols = {
    'total_payments_6m': 0,
    'total_late_15_6m': 0,
    'total_late_30_6m': 0,
    'total_late_60_6m': 0,
    'total_late_90_6m': 0,
    'max_days_late_6m': 0,
    'avg_days_late_6m': 0,
    'total_late_30_3m': 0,
    'total_late_90_3m': 0,
    'payment_discipline_flag': 'GOOD',
    'recent_payment_stress': 0
}
final_df = final_df.fillna(fill_cols)

# Convert types
final_df['recent_payment_stress'] = final_df['recent_payment_stress'].astype(int)

# ============================================================================
# STEP 10: VALIDATION & STATISTICS
# ============================================================================
print("\n" + "="*60)
print("✅ INSTALLMENT FEATURES COMPLETE!")
print("="*60)

print(f"\n📊 Dataset: {final_df.shape}")
print(f"   Total customers: {len(final_df):,}")
print(f"   With payment history: {(final_df['total_payments_6m'] > 0).sum():,}")

print(f"\n📈 Payment Discipline Distribution:")
for disc, count in final_df['payment_discipline_flag'].value_counts().items():
    print(f"   {disc:10s}: {count:6,} ({count/len(final_df)*100:5.1f}%)")

print(f"\n⚠️ Recent Payment Stress: {final_df['recent_payment_stress'].sum():,}")

print(f"\n🔢 Payment Statistics:")
print(final_df[['total_payments_6m', 'total_late_15_6m', 'total_late_30_6m', 
               'total_late_90_6m', 'avg_days_late_6m']].describe())

# ============================================================================
# STEP 11: SAVE
# ============================================================================
output_file = Path('DATA/processed/customer_bureau_installment_features.csv')
final_df.to_csv(output_file, index=False)

print(f"\n💾 Saved: {output_file}")
print(f"   Total columns: {final_df.shape[1]}")
print(f"   New columns added: {final_df.shape[1] - len(bureau_df.columns)}")

print("\n✅ Ready for Step 5: Cash Flow & Liquidity Analysis")
# ```

# ---

# ## 🎯 What This Step Does

# **Installment Payment Analysis** tracks:

# 1. **Days Late** — how many days past due for each payment
# 2. **Delay Buckets** — 15/30/60/90+ day delays
# 3. **Time Windows** — last 6 months & 3 months
# 4. **Payment Discipline Flag** — GOOD / MODERATE / POOR
# 5. **Recent Stress** — any 30+ day delays in last 3 months

# ---

# ## 📊 New Columns Added

# After this step, your dataset will have:

# **Previous 12 columns** +
# - `total_payments_6m`
# - `total_late_15_6m`
# - `total_late_30_6m`
# - `total_late_60_6m`
# - `total_late_90_6m`
# - `max_days_late_6m`
# - `avg_days_late_6m`
# - `total_late_30_3m`
# - `total_late_90_3m`
# - `payment_discipline_flag`
# - `recent_payment_stress`

# = **23 columns total**

# ---

# ## 🚀 Run This Now

# 1. Save as `notebooks/02_installment_features.ipynb`
# 2. Run all cells
# 3. Wait for completion (~1-2 minutes)

# Once complete, you'll see:
# ```
# ✅ Ready for Step 5: Cash Flow & Liquidity Analysis

🚀 Starting Installment Payment Analysis...

📂 Loading data...
✅ Bureau features: 307,511 customers
✅ Installment records: 13,605,401

⏰ Calculating payment delays...
   Payment delay range: -3189 to 2884 days

📊 Creating delay categories...

📅 Filtering recent payments...
   Last 6 months: 1,660,583 payments
   Last 3 months: 774,253 payments

🧮 Aggregating payment behavior per loan...
   Features created for 290,504 loans

🔗 Mapping loans to customers...

👤 Aggregating to customer level...
   207,651 customers with payment history

🚦 Creating payment discipline flags...

🔄 Merging with bureau features...

✅ INSTALLMENT FEATURES COMPLETE!

📊 Dataset: (307511, 23)
   Total customers: 307,511
   With payment history: 179,379

📈 Payment Discipline Distribution:
   GOOD      : 307,425 (100.0%)
   POOR      :     57 (  0.0%)
   MODERATE  :     29 (  0.0%)

⚠️ Recent Payment Stress: 248

🔢 Payment Statistics:
       total_payments_6m  total_late_15_6m  total_late_30_6m  \
count      307511.0

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path

print("🚀 Starting Cash Flow & Liquidity Analysis...")

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("\n📂 Loading data...")

# Load previous features
previous_df = pd.read_csv('DATA/processed/customer_bureau_installment_features.csv')

# Load balance data
credit_card = pd.read_csv('DATA/credit_card_balance.csv')
pos_cash = pd.read_csv('DATA/POS_CASH_balance.csv')

print(f"✅ Previous features: {len(previous_df):,} customers")
print(f"✅ Credit card records: {len(credit_card):,}")
print(f"✅ POS cash records: {len(pos_cash):,}")

# ============================================================================
# STEP 2: PROCESS CREDIT CARD BALANCES
# ============================================================================
print("\n💳 Processing credit card balances...")

# Filter last 6 months (MONTHS_BALANCE: 0 = most recent, -1 = 1 month ago)
cc_6m = credit_card[credit_card['MONTHS_BALANCE'] >= -6].copy()

# Aggregate per customer
cc_features = cc_6m.groupby('SK_ID_CURR').agg(
    # Balance metrics
    avg_balance_cc=(('AMT_BALANCE', 'mean')),
    total_drawings_cc=(('AMT_DRAWINGS_CURRENT', 'sum')),
    
    # Credit metrics
    avg_credit_limit=(('AMT_CREDIT_LIMIT_ACTUAL', 'mean')),
    
    # Utilization (balance / limit)
    max_utilization=(('AMT_BALANCE', lambda x: (x / cc_6m.loc[x.index, 'AMT_CREDIT_LIMIT_ACTUAL'].replace(0, 1)).max())),
    
    # Payment behavior
    total_payments_cc=(('AMT_PAYMENT_CURRENT', 'sum')),
    
    # Delinquency
    dpd_count_cc=(('SK_DPD', lambda x: (x > 0).sum()))
).reset_index()

print(f"   {len(cc_features):,} customers with credit card history")

# ============================================================================
# STEP 3: PROCESS POS/CASH BALANCES
# ============================================================================
print("\n🏪 Processing POS/cash balances...")

# Filter last 6 months
pos_6m = pos_cash[pos_cash['MONTHS_BALANCE'] >= -6].copy()

# Aggregate per customer
pos_features = pos_6m.groupby('SK_ID_CURR').agg(
    # Balance metrics
    avg_balance_pos=(('CNT_INSTALMENT_FUTURE', 'mean')),
    
    # Delinquency
    dpd_count_pos=(('SK_DPD', lambda x: (x > 0).sum()))
).reset_index()

print(f"   {len(pos_features):,} customers with POS/cash history")

# ============================================================================
# STEP 4: MAP TO CUSTOMER IDs
# ============================================================================
print("\n🔗 Mapping to customer IDs...")

# Get customer mapping from previous application
prev_app = pd.read_csv('DATA/previous_application.csv',
                       usecols=['SK_ID_PREV', 'SK_ID_CURR'])

# Already have SK_ID_CURR in cc and pos, so no mapping needed
# Just rename for consistency
cc_features = cc_features.rename(columns={'SK_ID_CURR': 'customer_id'})
pos_features = pos_features.rename(columns={'SK_ID_CURR': 'customer_id'})

# ============================================================================
# STEP 5: COMBINE BALANCE SOURCES
# ============================================================================
print("\n🔄 Combining balance sources...")

# Merge credit card and POS features
balance_features = cc_features.merge(pos_features, on='customer_id', how='outer')
balance_features = balance_features.fillna(0)

# Calculate combined metrics
balance_features['total_credit_activity'] = (
    balance_features['total_drawings_cc'] + 
    balance_features['total_payments_cc']
)

balance_features['total_dpd_count'] = (
    balance_features['dpd_count_cc'] + 
    balance_features['dpd_count_pos']
)

print(f"   {len(balance_features):,} customers with balance history")

# ============================================================================
# STEP 6: CREATE LIQUIDITY METRICS
# ============================================================================
print("\n💰 Creating liquidity metrics...")

# Get income from main application (already in previous_df)
# We'll calculate liquidity based on balance vs EMI

# For now, create placeholders that we'll fill after merge
balance_features['avg_monthly_balance_6m'] = balance_features['avg_balance_cc']

# ============================================================================
# STEP 7: MERGE WITH PREVIOUS FEATURES
# ============================================================================
print("\n🔄 Merging with previous features...")

final_df = previous_df.merge(balance_features, on='customer_id', how='left')

# Fill missing values
fill_cols = {
    'avg_balance_cc': 0,
    'total_drawings_cc': 0,
    'avg_credit_limit': 0,
    'max_utilization': 0,
    'total_payments_cc': 0,
    'dpd_count_cc': 0,
    'avg_balance_pos': 0,
    'dpd_count_pos': 0,
    'total_credit_activity': 0,
    'total_dpd_count': 0,
    'avg_monthly_balance_6m': 0
}
final_df = final_df.fillna(fill_cols)

# ============================================================================
# STEP 8: CALCULATE DERIVED LIQUIDITY FEATURES
# ============================================================================
print("\n📊 Calculating derived liquidity features...")

# Total EMI (if exists in previous_df)
if 'AMT_ANNUITY' in final_df.columns:
    final_df['total_emi_monthly'] = final_df['AMT_ANNUITY'].fillna(0)
else:
    final_df['total_emi_monthly'] = 0

# Net cash surplus approximation
# Credit activity - EMI obligations
final_df['net_cash_surplus_6m'] = (
    final_df['total_credit_activity'] - 
    (final_df['total_emi_monthly'] * 6)
)

# Calculate total credits and debits (6 months)
final_df['total_credit_6m'] = final_df['total_drawings_cc'] * 6
final_df['total_debit_6m'] = final_df['total_payments_cc'] * 6

# Liquidity flag
def assign_liquidity_flag(row):
    if row['total_emi_monthly'] == 0:
        return 'ADEQUATE'  # No EMI burden
    
    balance_coverage = row['avg_monthly_balance_6m'] / row['total_emi_monthly']
    
    if balance_coverage >= 1.5:
        return 'ADEQUATE'
    elif balance_coverage >= 1.0:
        return 'MODERATE'
    else:
        return 'LOW'

final_df['liquidity_flag'] = final_df.apply(assign_liquidity_flag, axis=1)

# ============================================================================
# STEP 9: CREATE CASH FLOW HEALTH FLAG
# ============================================================================
print("\n🚦 Creating cash flow health flags...")

def assign_cashflow_health(row):
    if row['net_cash_surplus_6m'] > 0 and row['total_dpd_count'] == 0:
        return 'HEALTHY'
    elif row['net_cash_surplus_6m'] > 0 and row['total_dpd_count'] <= 2:
        return 'STABLE'
    elif row['net_cash_surplus_6m'] <= 0 and row['total_dpd_count'] > 2:
        return 'STRESSED'
    else:
        return 'MODERATE'

final_df['cashflow_health'] = final_df.apply(assign_cashflow_health, axis=1)

# Inward bounce count (proxy using DPD)
final_df['inward_bounce_count_3m'] = (final_df['total_dpd_count'] > 2).astype(int)

# ============================================================================
# STEP 10: VALIDATION & STATISTICS
# ============================================================================
print("\n" + "="*60)
print("✅ CASH FLOW & LIQUIDITY FEATURES COMPLETE!")
print("="*60)

print(f"\n📊 Dataset: {final_df.shape}")
print(f"   Total customers: {len(final_df):,}")
print(f"   With balance history: {(final_df['avg_monthly_balance_6m'] > 0).sum():,}")

print(f"\n💰 Liquidity Distribution:")
for liq, count in final_df['liquidity_flag'].value_counts().items():
    print(f"   {liq:10s}: {count:6,} ({count/len(final_df)*100:5.1f}%)")

print(f"\n📈 Cash Flow Health Distribution:")
for health, count in final_df['cashflow_health'].value_counts().items():
    print(f"   {health:10s}: {count:6,} ({count/len(final_df)*100:5.1f}%)")

print(f"\n🔢 Balance Statistics:")
print(final_df[['avg_monthly_balance_6m', 'total_credit_6m', 'total_debit_6m',
               'net_cash_surplus_6m']].describe())

# ============================================================================
# STEP 11: SAVE
# ============================================================================
output_file = Path('DATA/processed/customer_complete_features.csv')
final_df.to_csv(output_file, index=False)

print(f"\n💾 Saved: {output_file}")
print(f"   Total columns: {final_df.shape[1]}")
print(f"   Columns added: {final_df.shape[1] - len(previous_df.columns)}")

print("\n✅ Ready for Step 6: Synthetic Salary Features")

🚀 Starting Cash Flow & Liquidity Analysis...

📂 Loading data...
✅ Previous features: 307,511 customers
✅ Credit card records: 3,840,312
✅ POS cash records: 10,001,358

💳 Processing credit card balances...
   103,556 customers with credit card history

🏪 Processing POS/cash balances...
   192,077 customers with POS/cash history

🔗 Mapping to customer IDs...

🔄 Combining balance sources...
   231,257 customers with balance history

💰 Creating liquidity metrics...

🔄 Merging with previous features...

📊 Calculating derived liquidity features...

🚦 Creating cash flow health flags...

✅ CASH FLOW & LIQUIDITY FEATURES COMPLETE!

📊 Dataset: (307511, 41)
   Total customers: 307,511
   With balance history: 34,530

💰 Liquidity Distribution:
   LOW       : 278,038 ( 90.4%)
   ADEQUATE  : 27,987 (  9.1%)
   MODERATE  :  1,486 (  0.5%)

📈 Cash Flow Health Distribution:
   MODERATE  : 285,748 ( 92.9%)
   HEALTHY   : 18,911 (  6.1%)
   STRESSED  :  2,098 (  0.7%)
   STABLE    :    754 (  0.2%)

🔢 Ba